In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = "/content/drive/MyDrive/Project/dataset"
os.listdir(path)

['test', 'train']

In [4]:
train_path = os.path.join(path, "train")
print(os.listdir(train_path))
print("*"*100)
test_path = os.path.join(path, "test")
print(os.listdir(test_path))

['woman', 'man']
****************************************************************************************************
['woman', 'man']


In [5]:
from glob import glob
folders = glob("/content/drive/MyDrive/Project/dataset/train/*")
folders

['/content/drive/MyDrive/Project/dataset/train/woman',
 '/content/drive/MyDrive/Project/dataset/train/man']

In [6]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

In [7]:
SIZE = [224, 224]

In [8]:
from tensorflow.keras.applications.vgg19 import VGG19

In [9]:
vg19 = VGG19(input_shape=SIZE + [3], weights="imagenet", include_top=False)

80142336/80134624 [==============================] - 1s 0us/step


In [10]:
for layer in vg19.layers:
    layer.trainable = False

In [11]:
x = Flatten()(vg19.output)

In [12]:
prediction = Dense(len(folders), activation="softmax")(x)

modelvg = Model(inputs=vg19.input, outputs=prediction)

In [13]:
modelvg.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [14]:
modelvg.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [15]:
train_datagen_vg19 = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen_vg19 = ImageDataGenerator(rescale=1./255)

In [16]:
trainning_set_vg19 = train_datagen_vg19.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=10,
                                                 class_mode="categorical")

Found 561 images belonging to 2 classes.


In [17]:
testing_set_vg19 = test_datagen_vg19.flow_from_directory(test_path,
                                                 target_size=(224, 224),
                                                 batch_size=10,
                                                 class_mode="categorical")

Found 340 images belonging to 2 classes.


In [18]:
r_vg19 = modelvg.fit_generator(trainning_set_vg19,
                       validation_data=testing_set_vg19,
                       epochs=10,
                       steps_per_epoch=len(trainning_set_vg19),
                       validation_steps=len(testing_set_vg19)
                       )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
57/57 [==============================] - 492s 9s/step - loss: 0.4709 - accuracy: 0.8039 - val_loss: 0.3999 - val_accuracy: 0.8353
Epoch 2/10
57/57 [==============================] - 10s 181ms/step - loss: 0.1584 - accuracy: 0.9287 - val_loss: 0.2950 - val_accuracy: 0.8735
Epoch 3/10
57/57 [==============================] - 10s 181ms/step - loss: 0.2096 - accuracy: 0.9020 - val_loss: 0.5891 - val_accuracy: 0.8324
Epoch 4/10
57/57 [==============================] - 10s 182ms/step - loss: 0.1330 - accuracy: 0.9554 - val_loss: 0.2843 - val_accuracy: 0.8824
Epoch 5/10
57/57 [==============================] - 11s 186ms/step - loss: 0.0529 - accuracy: 0.9768 - val_loss: 0.3317 - val_accuracy: 0.8853
Epoch 6/10
57/57 [==============================] - 11s 186ms/step - loss: 0.0585 - accuracy: 0.9822 - val_loss: 0.2561 - val_accuracy: 0.9059
Epoch 7/10
57/57 [==============================] - 11s 187ms/step -

In [19]:
modelvg.save("/content/drive/MyDrive/Project/mymodel.model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/mymodel.model/assets


In [20]:
import cv2
import tensorflow as tf
def prepare(filepath):
  img_array = cv2.imread(filepath,cv2.IMREAD_COLOR)
  img_array = img_array/255
  new_array = cv2.resize(img_array,(224,224))
  return new_array.reshape(-1,224,224,3)

model = tf.keras.models.load_model("/content/drive/MyDrive/Project/mymodel.model")

In [28]:
prediction = model.predict([prepare("/content/car.jpg")])
x = np.argmax(prediction)
print(x)

1


In [29]:
if x==0:
  print("male")
else:
  print("female")

female
